In [1]:
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.callbacks import ReduceLROnPlateau
import os

from keras.backend import image_data_format
from keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalMaxPooling2D, AveragePooling2D, concatenate, GlobalAveragePooling2D, Dense
from keras.models import Model
from tensorflow.keras.utils import get_file
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf


In [2]:
BATCH_SIZE = 32
EPOCHS = 20
IMAGE_SIZE = (224, 224)

In [3]:
def crea(img):
  if(img==0):
    return 0
  elif(img==1):
    return 1
  elif(img==2):
    return 2
  elif(img==3):
    return 3
  elif(img==4):
    return 4
  else:
    return 5
data=pd.read_csv('../input/ci-sc22-places-and-scene-recognition/train.csv')
filenames = tf.io.gfile.glob('../input/ci-sc22-places-and-scene-recognition/train_images/train_images/*')
image_data_train = pd.DataFrame(data={'filename': filenames, 'class': [crea(np.array(data['label'][data['image_name'] == x.split('/')[-1]])) for x in filenames]})


In [4]:
def labele_image(df):
    images = []
    for file in df['filename']:
        image = cv2.imread(file)
        image = cv2.resize(image,IMAGE_SIZE)
        images.append(image)
    images = np.array(images)
    
    labels = df.loc[:, 'class']
    return images, labels

In [5]:
image_data_train=shuffle(image_data_train)

print('Train sample: ', len(image_data_train['class']), dict(image_data_train['class'].value_counts()))


Train sample:  13627 {3: 2430, 2: 2366, 5: 2306, 4: 2227, 1: 2196, 0: 2102}


In [6]:
image_data_val=image_data_train.sample(3000)
train_images, train_labels = labele_image(image_data_train)
val_images, val_labels = labele_image(image_data_val)

print(f'Shape of train set: {train_images.shape}')
print(f'Shape of train set: {train_labels.shape}')
print(f'Shape of train set: {val_images.shape}')
print(f'Shape of train set: {val_labels.shape}')

Shape of train set: (13627, 224, 224, 3)
Shape of train set: (13627,)
Shape of train set: (3000, 224, 224, 3)
Shape of train set: (3000,)


In [7]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
from functools import partial

from keras.models import Model
from keras.layers import Activation
from keras.layers import AveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import MaxPooling2D
from keras.utils.data_utils import get_file
from keras import backend as K


BASE_WEIGHT_URL = 'https://github.com/myutwo150/keras-inception-resnet-v2/releases/download/v0.1/'


def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x


def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    """Utility function to apply conv + BN.
    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        kernel_size: kernel size as in `Conv2D`.
        padding: padding mode in `Conv2D`.
        activation: activation in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_Activation'`
            for the activation and `name + '_BatchNorm'` for the
            batch norm layer.
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    x = Conv2D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = _generate_layer_name('BatchNorm', prefix=name)
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation is not None:
        ac_name = _generate_layer_name('Activation', prefix=name)
        x = Activation(activation, name=ac_name)(x)
    return x


def _generate_layer_name(name, branch_idx=None, prefix=None):
    """Utility function for generating layer names.
    If `prefix` is `None`, returns `None` to use default automatic layer names.
    Otherwise, the returned layer name is:
        - PREFIX_NAME if `branch_idx` is not given.
        - PREFIX_Branch_0_NAME if e.g. `branch_idx=0` is given.
    # Arguments
        name: base layer name string, e.g. `'Concatenate'` or `'Conv2d_1x1'`.
        branch_idx: an `int`. If given, will add e.g. `'Branch_0'`
            after `prefix` and in front of `name` in order to identify
            layers in the same block but in different branches.
        prefix: string prefix that will be added in front of `name` to make
            all layer names unique (e.g. which block this layer belongs to).
    # Returns
        The layer name.
    """
    if prefix is None:
        return None
    if branch_idx is None:
        return '_'.join((prefix, name))
    return '_'.join((prefix, 'Branch', str(branch_idx), name))


def _inception_resnet_block(x, scale, block_type, block_idx, activation='relu'):
    """Adds a Inception-ResNet block.
    This function builds 3 types of Inception-ResNet blocks mentioned
    in the paper, controlled by the `block_type` argument (which is the
    block name used in the official TF-slim implementation):
        - Inception-ResNet-A: `block_type='Block35'`
        - Inception-ResNet-B: `block_type='Block17'`
        - Inception-ResNet-C: `block_type='Block8'`
    # Arguments
        x: input tensor.
        scale: scaling factor to scale the residuals before adding
            them to the shortcut branch.
        block_type: `'Block35'`, `'Block17'` or `'Block8'`, determines
            the network structure in the residual branch.
        block_idx: used for generating layer names.
        activation: name of the activation function to use at the end
            of the block (see [activations](../activations.md)).
            When `activation=None`, no activation is applied
            (i.e., "linear" activation: `a(x) = x`).
    # Returns
        Output tensor for the block.
    # Raises
        ValueError: if `block_type` is not one of `'Block35'`,
            `'Block17'` or `'Block8'`.
    """
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    if block_idx is None:
        prefix = None
    else:
        prefix = '_'.join((block_type, str(block_idx)))
    name_fmt = partial(_generate_layer_name, prefix=prefix)

    if block_type == 'Block35':
        branch_0 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 32, 3, name=name_fmt('Conv2d_0b_3x3', 1))
        branch_2 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 2))
        branch_2 = conv2d_bn(branch_2, 48, 3, name=name_fmt('Conv2d_0b_3x3', 2))
        branch_2 = conv2d_bn(branch_2, 64, 3, name=name_fmt('Conv2d_0c_3x3', 2))
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Block17':
        branch_0 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 160, [1, 7], name=name_fmt('Conv2d_0b_1x7', 1))
        branch_1 = conv2d_bn(branch_1, 192, [7, 1], name=name_fmt('Conv2d_0c_7x1', 1))
        branches = [branch_0, branch_1]
    elif block_type == 'Block8':
        branch_0 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 224, [1, 3], name=name_fmt('Conv2d_0b_1x3', 1))
        branch_1 = conv2d_bn(branch_1, 256, [3, 1], name=name_fmt('Conv2d_0c_3x1', 1))
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "Block35", "Block17" or "Block8", '
                         'but got: ' + str(block_type))

    mixed = Concatenate(axis=channel_axis, name=name_fmt('Concatenate'))(branches)
    up = conv2d_bn(mixed,
                   K.int_shape(x)[channel_axis],
                   1,
                   activation=None,
                   use_bias=True,
                   name=name_fmt('Conv2d_1x1'))
    x = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
               output_shape=K.int_shape(x)[1:],
               arguments={'scale': scale},
               name=name_fmt('ScaleSum'))([x, up])
    if activation is not None:
        x = Activation(activation, name=name_fmt('Activation'))(x)
    return x


def InceptionResNetV2(include_top=True,
                      weights='imagenet',
                      input_tensor=None,
                      input_shape=None,
                      pooling=None,
                      classes=1000,
                      dropout_keep_prob=0.8):
  
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    # Stem block: 35 x 35 x 192
    x = conv2d_bn(img_input, 32, 3, strides=2, padding='valid', name='Conv2d_1a_3x3')
    x = conv2d_bn(x, 32, 3, padding='valid', name='Conv2d_2a_3x3')
    x = conv2d_bn(x, 64, 3, name='Conv2d_2b_3x3')
    x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
    x = conv2d_bn(x, 80, 1, padding='valid', name='Conv2d_3b_1x1')
    x = conv2d_bn(x, 192, 3, padding='valid', name='Conv2d_4a_3x3')
    x = MaxPooling2D(3, strides=2, name='MaxPool_5a_3x3')(x)

    # Mixed 5b (Inception-A block): 35 x 35 x 320
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    name_fmt = partial(_generate_layer_name, prefix='Mixed_5b')
    branch_0 = conv2d_bn(x, 96, 1, name=name_fmt('Conv2d_1x1', 0))
    branch_1 = conv2d_bn(x, 48, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1, 64, 5, name=name_fmt('Conv2d_0b_5x5', 1))
    branch_2 = conv2d_bn(x, 64, 1, name=name_fmt('Conv2d_0a_1x1', 2))
    branch_2 = conv2d_bn(branch_2, 96, 3, name=name_fmt('Conv2d_0b_3x3', 2))
    branch_2 = conv2d_bn(branch_2, 96, 3, name=name_fmt('Conv2d_0c_3x3', 2))
    branch_pool = AveragePooling2D(3,
                                   strides=1,
                                   padding='same',
                                   name=name_fmt('AvgPool_0a_3x3', 3))(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, name=name_fmt('Conv2d_0b_1x1', 3))
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_5b')(branches)

    # 10x Block35 (Inception-ResNet-A block): 35 x 35 x 320
    for block_idx in range(1, 11):
        x = _inception_resnet_block(x,
                                    scale=0.17,
                                    block_type='Block35',
                                    block_idx=block_idx)

    # Mixed 6a (Reduction-A block): 17 x 17 x 1088
    name_fmt = partial(_generate_layer_name, prefix='Mixed_6a')
    branch_0 = conv2d_bn(x,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1, 256, 3, name=name_fmt('Conv2d_0b_3x3', 1))
    branch_1 = conv2d_bn(branch_1,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 2))(x)
    branches = [branch_0, branch_1, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_6a')(branches)

    # 20x Block17 (Inception-ResNet-B block): 17 x 17 x 1088
    for block_idx in range(1, 21):
        x = _inception_resnet_block(x,
                                    scale=0.1,
                                    block_type='Block17',
                                    block_idx=block_idx)

    # Mixed 7a (Reduction-B block): 8 x 8 x 2080
    name_fmt = partial(_generate_layer_name, prefix='Mixed_7a')
    branch_0 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 0))
    branch_0 = conv2d_bn(branch_0,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1,
                         288,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_2 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 2))
    branch_2 = conv2d_bn(branch_2, 288, 3, name=name_fmt('Conv2d_0b_3x3', 2))
    branch_2 = conv2d_bn(branch_2,
                         320,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 2))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 3))(x)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_7a')(branches)

    # 10x Block8 (Inception-ResNet-C block): 8 x 8 x 2080
    for block_idx in range(1, 10):
        x = _inception_resnet_block(x,
                                    scale=0.2,
                                    block_type='Block8',
                                    block_idx=block_idx)
    x = _inception_resnet_block(x,
                                scale=1.,
                                activation=None,
                                block_type='Block8',
                                block_idx=10)

    # Final convolution block
    x = conv2d_bn(x, 1536, 1, name='Conv2d_7b_1x1')

    if include_top:
        # Classification block
        x = GlobalAveragePooling2D(name='AvgPool')(x)
        x = Dropout(1.0 - dropout_keep_prob, name='Dropout')(x)
        x = Dense(classes, name='Logits')(x)
        x = Activation('softmax', name='Predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D(name='AvgPool')(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D(name='MaxPool')(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`
    if input_tensor is not None:
        inputs = tf.keras.utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model
    model = Model(inputs, x, name='inception_resnet_v2')

    # Load weights
    if weights == 'imagenet':
        if K.image_data_format() == 'channels_first':
            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
        if include_top:
            weights_filename = 'inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5'
            weights_path = get_file(weights_filename,
                                    BASE_WEIGHT_URL + weights_filename,
                                    cache_subdir='models',
                                    md5_hash='e693bd0210a403b3192acc6073ad2e96')
        else:
            weights_filename = 'inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
            weights_path = get_file(weights_filename,
                                    BASE_WEIGHT_URL + weights_filename,
                                    cache_subdir='models',
                                    md5_hash='d19885ff4a710c122648d3b5c3b684e4')
        model.load_weights(weights_path)

    return model

In [8]:
model = InceptionResNetV2(include_top=False,
                      weights='imagenet',
                      input_shape=(224,224,3),
                      pooling=None,
                      classes=1000,
                      dropout_keep_prob=0.8)

2021-12-28 23:01:35.103509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-28 23:01:35.224955: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-28 23:01:35.226149: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-28 23:01:35.227953: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

219070464/219055592 [==============================] - 9s 0us/step


In [9]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(6, activation='softmax')(x)


model = Model(inputs=model.input, outputs=predictions)

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
init_time = datetime.datetime.now()
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 1, verbose=1, factor=0.3, min_lr=0.000001)

trained = model.fit(
                    train_images, train_labels,
                    validation_data = (val_images, val_labels),
                    batch_size = BATCH_SIZE, 
                    epochs=14,
                    callbacks=[learning_rate_reduction]
    )

2021-12-28 23:01:53.776747: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2051245056 exceeds 10% of free system memory.
2021-12-28 23:01:55.793628: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2051245056 exceeds 10% of free system memory.
2021-12-28 23:01:57.464912: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/14


2021-12-28 23:02:18.533434: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


426/426 [==============================] - 182s 362ms/step - loss: 0.4621 - accuracy: 0.8492 - val_loss: 1.2292 - val_accuracy: 0.6943
Epoch 2/14
426/426 [==============================] - 148s 348ms/step - loss: 0.3116 - accuracy: 0.8951 - val_loss: 0.2392 - val_accuracy: 0.9160
Epoch 3/14
426/426 [==============================] - 148s 346ms/step - loss: 0.2596 - accuracy: 0.9121 - val_loss: 0.7206 - val_accuracy: 0.8020

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 4/14
426/426 [==============================] - 148s 347ms/step - loss: 0.1481 - accuracy: 0.9483 - val_loss: 0.0871 - val_accuracy: 0.9707
Epoch 5/14
426/426 [==============================] - 148s 348ms/step - loss: 0.1067 - accuracy: 0.9621 - val_loss: 0.0438 - val_accuracy: 0.9867
Epoch 6/14
426/426 [==============================] - 150s 352ms/step - loss: 0.0798 - accuracy: 0.9723 - val_loss: 0.0572 - val_accuracy: 0.9763

Epoch 00006: ReduceLROnPlateau reducing learning rate

In [11]:
model.save('modelincenAet1.h5')

In [12]:
WEIGHTS_PATH = (
    'https://github.com/fchollet/deep-learning-models/'
    'releases/download/v0.5/'
    'inception_v3_weights_tf_dim_ordering_tf_kernels.h5')
WEIGHTS_PATH_NO_TOP = (
    'https://github.com/fchollet/deep-learning-models/'
    'releases/download/v0.5/'
    'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
def load_weights(model,weights,include_top):
    if weights == 'imagenet':
        if include_top:
            weights_path =  tf.keras.utils.get_file(
                'inception_v3_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,
                cache_subdir='models',
                file_hash='9a0d58056eeedaa3f26cb7ebd46da564')
        else:
            weights_path =  tf.keras.utils.get_file(
                'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,
                cache_subdir='models',
                file_hash='bcbd6486424b2319ff4ef7d526e38f63')
        model.load_weights(weights_path)
    elif weights is not None:
        model.load_weights(weights)
    
    return model
    

def makelayer(x,filters,num_row,num_col,padding='same',strides=(1, 1),name=None):

   
    if image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(filters, (num_row, num_col),strides=strides,padding=padding,use_bias=False,name=name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=name)(x)
    x = Activation('relu', name=name)(x)
    return x
def blockA(x,unique_filters,concat_axis,name=None):
    branch1x1 = makelayer(x, 64, 1, 1)

    branch5x5 = makelayer(x, 48, 1, 1)
    branch5x5 = makelayer(branch5x5, 64, 5, 5)

    branch3x3dbl = makelayer(x, 64, 1, 1)
    branch3x3dbl = makelayer(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = makelayer(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3),
                                          strides=(1, 1),
                                          padding='same')(x)
    branch_pool = makelayer(branch_pool, unique_filters, 1, 1)
    x = concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=concat_axis,
        name='mixed'+name)
    return x

def blockB(x,unique_filters,concat_axis,name=None):
    branch1x1 = makelayer(x, 192, 1, 1)

    branch7x7 = makelayer(x, unique_filters, 1, 1)
    branch7x7 = makelayer(branch7x7, unique_filters, 1, 7)
    branch7x7 = makelayer(branch7x7, 192, 7, 1)

    branch7x7dbl = makelayer(x, unique_filters, 1, 1)
    branch7x7dbl = makelayer(branch7x7dbl, unique_filters, 7, 1)
    branch7x7dbl = makelayer(branch7x7dbl, unique_filters, 1, 7)
    branch7x7dbl = makelayer(branch7x7dbl, unique_filters, 7, 1)
    branch7x7dbl = makelayer(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = makelayer(branch_pool, 192, 1, 1)
    x = concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=concat_axis,
        name='mixed' + name)
    return x

def blockC(x,concat_axis,name=None):
    branch1x1 = makelayer(x, 320, 1, 1)

    branch3x3 = makelayer(x, 384, 1, 1)
    branch3x3_1 = makelayer(branch3x3, 384, 1, 3)
    branch3x3_2 = makelayer(branch3x3, 384, 3, 1)
    branch3x3 = concatenate(
        [branch3x3_1, branch3x3_2],
        axis=concat_axis,
        name='mixed9_' + name)

    branch3x3dbl = makelayer(x, 448, 1, 1)
    branch3x3dbl = makelayer(branch3x3dbl, 384, 3, 3)
    branch3x3dbl_1 = makelayer(branch3x3dbl, 384, 1, 3)
    branch3x3dbl_2 = makelayer(branch3x3dbl, 384, 3, 1)
    branch3x3dbl = concatenate(
        [branch3x3dbl_1, branch3x3dbl_2], axis=concat_axis)

    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = makelayer(branch_pool, 192, 1, 1)
    x = concatenate(
        [branch1x1, branch3x3, branch3x3dbl, branch_pool],
        axis=concat_axis,
        name='mixed' + str(9 + int(name)))
    return x
def auxiliary_classifier(x,num_classes):
    x1 = AveragePooling2D((5,5),strides=(3,3))(x)
    x1 = makelayer(x1, 128,1,1)
    x1 = Dense(1000, activation='softmax',name='auxillary_classifier')(x1)
    return x1
def InceptionV3(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000,
                **kwargs):

    
    img_input = input_tensor

    if image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3

    x = makelayer(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = makelayer(x, 32, 3, 3, padding='valid')
    x = makelayer(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = makelayer(x, 80, 1, 1, padding='valid')
    x = makelayer(x, 192, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #blockA divide 5x5 con to 2*(3X3)
    x = blockA(x,32,channel_axis,"0")
    x = blockA(x,64,channel_axis,"1")
    x = blockA(x,64,channel_axis,"2")


   
    branch3x3 = makelayer(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = makelayer(x, 64, 1, 1)
    branch3x3dbl = makelayer(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = makelayer(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate(
        [branch3x3, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed3')
    #blockB divide 7X7 to 2 asym 1X7,7X1

    x = blockB(x,128,channel_axis,"4")
    x = blockB(x,160,channel_axis,"5")
    x = blockB(x,160,channel_axis,"6")
    x =blockB(x,192,channel_axis,"7")


    x1 = auxiliary_classifier(x,classes)

 
    branch3x3 = makelayer(x, 192, 1, 1)
    branch3x3 = makelayer(branch3x3, 320, 3, 3,
                          strides=(2, 2), padding='valid')

    branch7x7x3 = makelayer(x, 192, 1, 1)
    branch7x7x3 = makelayer(branch7x7x3, 192, 1, 7)
    branch7x7x3 = makelayer(branch7x7x3, 192, 7, 1)
    branch7x7x3 = makelayer(
        branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = concatenate(
        [branch3x3, branch7x7x3, branch_pool],
        axis=channel_axis,
        name='mixed8')

    #blockC divide 3X3 to 2 asym 1X3,3X1
    x = blockC(x,channel_axis,"1")
    x = blockC(x,channel_axis,"2")


    if include_top:
        # Classification block
        x = GlobalAveragePooling2D(name='avg_pool')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
    

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = tf.keras.utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    
    # Create model.
    model = Model(inputs, x, name='inception_v3')

    # Load weights.
    model = load_weights(model,weights,include_top)
    print("Inception v3 model created")

    return model

In [13]:
input_tensor = Input(shape=(224, 224, 3))
model2 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 2s 0us/step
Inception v3 model created


In [14]:
x = model2.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(6, activation='softmax')(x)

In [15]:



model2 = Model(inputs=model2.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
# for layer in model.layers:
#     layer.trainable = False

# compile the model (should be done after setting layers to non-trainable)
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
init_time = datetime.datetime.now()
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 1, verbose=1, factor=0.3, min_lr=0.000001)

trained = model2.fit(
                    train_images, train_labels,
                    validation_data = (val_images, val_labels),
                    batch_size = BATCH_SIZE, 
                    epochs=15,
                    callbacks=[learning_rate_reduction]
    )

2021-12-28 23:38:10.323339: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2051245056 exceeds 10% of free system memory.
2021-12-28 23:38:12.401830: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2051245056 exceeds 10% of free system memory.


Epoch 1/15
426/426 [==============================] - 77s 162ms/step - loss: 0.5702 - accuracy: 0.8160 - val_loss: 0.9231 - val_accuracy: 0.7020
Epoch 2/15
426/426 [==============================] - 65s 153ms/step - loss: 0.3761 - accuracy: 0.8708 - val_loss: 0.3963 - val_accuracy: 0.8657
Epoch 3/15
426/426 [==============================] - 65s 153ms/step - loss: 0.3153 - accuracy: 0.8930 - val_loss: 1.3865 - val_accuracy: 0.7507

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 4/15
426/426 [==============================] - 65s 152ms/step - loss: 0.1907 - accuracy: 0.9319 - val_loss: 0.1106 - val_accuracy: 0.9610
Epoch 5/15
426/426 [==============================] - 66s 154ms/step - loss: 0.1425 - accuracy: 0.9504 - val_loss: 0.1763 - val_accuracy: 0.9307

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
Epoch 6/15
426/426 [==============================] - 65s 152ms/step - loss: 0.0797 - accuracy: 0.9731 - val_loss

In [16]:
model2.save('inceptionv3.h5')

In [17]:
CLASSES = {'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5, 'forest': 1, 'buildings': 0}
filenames = tf.io.gfile.glob('/kaggle/input/intel-image-classification/seg_test/seg_test/*/*')
image_path_df_test = pd.DataFrame(data={'filename': filenames, 'class': [x.split('/')[-2] for x in filenames]})

print('Test sample: ', len(image_path_df_test['class']), dict(image_path_df_test['class'].value_counts()))


Test sample:  3000 {'glacier': 553, 'mountain': 525, 'sea': 510, 'street': 501, 'forest': 474, 'buildings': 437}


In [18]:
def shuffle_prune(df, BATCH_SIZE):
    df = shuffle(df, random_state=42)
    df.reset_index(drop=True, inplace=True)
    df = df[ : df.shape[0] // BATCH_SIZE * BATCH_SIZE]
    return df

In [19]:
image_path_df_test, image_path_df_val1  = train_test_split(image_path_df_test, test_size=0.5, random_state=42, stratify=image_path_df_test['class'])
image_path_df_test = shuffle_prune(image_path_df_test, BATCH_SIZE)
image_path_df_test['class'] = image_path_df_test['class'].map(CLASSES)

# image_path_df_val = shuffle_prune(image_path_df_val, BATCH_SIZE)
# image_path_df_val['class'] = image_path_df_val['class'].map(CLASSES)

print('Test sample: ', len(image_path_df_test['class']), dict(image_path_df_test['class'].value_counts()))
# print('Val  sample: ', len(image_path_df_val['class']), dict(image_path_df_val['class'].value_counts()))

Test sample:  1472 {2: 271, 3: 259, 4: 250, 5: 246, 1: 235, 0: 211}


In [20]:
# val_images, val_labels = get_images_and_labels_arrays(image_path_df_val)

# print(f'Shape of validation set: {val_images.shape}')
# print(f'Shape of validation set: {val_labels.shape}')
test_images, test_labels = labele_image(image_path_df_test)
test_images=np.array(test_images)
print(f'Shape of test set: {test_images.shape}')
print(f'Shape of test set: {test_labels.shape}')

Shape of test set: (1472, 224, 224, 3)
Shape of test set: (1472,)


In [21]:
# Load the model
keras_model = tf.keras.models.load_model('./modelincenAet1.h5', compile=False)
keras_model._name = 'modelincenet.h5'
keras_model2 = tf.keras.models.load_model('./inceptionv3.h5', compile=False)
keras_model2._name = 'modelincev3'
models = [keras_model, keras_model2]
#model_input = tf.keras.Input(shape=(125, 125, 3))
model_input = tf.keras.Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)  

ensemble_model.save('ensemble.h5')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [22]:
ensemble_model.compile(optimizer='adam', 
                      loss='sparse_categorical_crossentropy', 
                      metrics=['accuracy'])
test_loss, test_acc = ensemble_model.evaluate(test_images, test_labels , verbose=0)
print('\naccuracy:', test_acc, '  loss: ',test_loss)


accuracy: 0.9864130616188049   loss:  0.044471465051174164


In [23]:
filenames = tf.io.gfile.glob('../input/ci-sc22-places-and-scene-recognition/test_images/test_images/*')

image_path_df_predict = pd.DataFrame(data={"filename":filenames ,'imgname':  [x.split('/')[-1] for x in filenames], 'class': np.nan})
print(f'Number filenames: {len(image_path_df_predict)}')

Number filenames: 3407


In [24]:
to_predict_images, to_predict_labels = labele_image(image_path_df_predict)
to_predict_images=np.array(to_predict_images)

print(f'Shape of images set to prediction: {to_predict_images.shape}')

Shape of images set to prediction: (3407, 224, 224, 3)


In [25]:
predict = np.argmax(ensemble_model.predict(to_predict_images), axis=1)
predict

array([0, 0, 2, ..., 1, 1, 3])

In [26]:
import csv
row = []
fields = ['image_name', 'label']
for i in range(len(predict)):
    row .append([image_path_df_predict['imgname'][i],predict[i]])
with open('sample_submission5555.csv', 'w') as f:
    write = csv.writer(f)   
    write.writerow(fields)
    write.writerows(row)

In [27]:

# from __future__ import print_function
# from __future__ import absolute_import

# import warnings
# import numpy as np

# from keras.preprocessing import image

# from keras.models import Model
# from keras import layers
# from keras.layers import Dense
# from keras.layers import Input
# from keras.layers import BatchNormalization
# from keras.layers import Activation
# from keras.layers import Conv2D
# from keras.layers import SeparableConv2D
# from keras.layers import MaxPooling2D
# from keras.layers import GlobalAveragePooling2D
# from keras.layers import GlobalMaxPooling2D
# from keras.utils.data_utils import get_file
# from keras import backend as K
# from keras.applications.imagenet_utils import decode_predictions



# TF_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights_tf_dim_ordering_tf_kernels.h5'
# TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'


# def Xception(include_top=True, weights='imagenet',
#              input_tensor=None, input_shape=None,
#              pooling=None,
#              classes=1000):
#     """Instantiates the Xception architecture.
#     Optionally loads weights pre-trained
#     on ImageNet. This model is available for TensorFlow only,
#     and can only be used with inputs following the TensorFlow
#     data format `(width, height, channels)`.
#     You should set `image_data_format="channels_last"` in your Keras config
#     located at ~/.keras/keras.json.
#     Note that the default input image size for this model is 299x299.
#     # Arguments
#         include_top: whether to include the fully-connected
#             layer at the top of the network.
#         weights: one of `None` (random initialization)
#             or "imagenet" (pre-training on ImageNet).
#         input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
#             to use as image input for the model.
#         input_shape: optional shape tuple, only to be specified
#             if `include_top` is False (otherwise the input shape
#             has to be `(299, 299, 3)`.
#             It should have exactly 3 inputs channels,
#             and width and height should be no smaller than 71.
#             E.g. `(150, 150, 3)` would be one valid value.
#         pooling: Optional pooling mode for feature extraction
#             when `include_top` is `False`.
#             - `None` means that the output of the model will be
#                 the 4D tensor output of the
#                 last convolutional layer.
#             - `avg` means that global average pooling
#                 will be applied to the output of the
#                 last convolutional layer, and thus
#                 the output of the model will be a 2D tensor.
#             - `max` means that global max pooling will
#                 be applied.
#         classes: optional number of classes to classify images
#             into, only to be specified if `include_top` is True, and
#             if no `weights` argument is specified.
#     # Returns
#         A Keras model instance.
#     # Raises
#         ValueError: in case of invalid argument for `weights`,
#             or invalid input shape.
#         RuntimeError: If attempting to run this model with a
#             backend that does not support separable convolutions.
#     """
#     if weights not in {'imagenet', None}:
#         raise ValueError('The `weights` argument should be either '
#                          '`None` (random initialization) or `imagenet` '
#                          '(pre-training on ImageNet).')

#     if weights == 'imagenet' and include_top and classes != 1000:
#         raise ValueError('If using `weights` as imagenet with `include_top`'
#                          ' as true, `classes` should be 1000')

#     if K.backend() != 'tensorflow':
#         raise RuntimeError('The Xception model is only available with '
#                            'the TensorFlow backend.')
#     if K.image_data_format() != 'channels_last':
#         warnings.warn('The Xception model is only available for the '
#                       'input data format "channels_last" '
#                       '(width, height, channels). '
#                       'However your settings specify the default '
#                       'data format "channels_first" (channels, width, height). '
#                       'You should set `image_data_format="channels_last"` in your Keras '
#                       'config located at ~/.keras/keras.json. '
#                       'The model being returned right now will expect inputs '
#                       'to follow the "channels_last" data format.')
#         K.set_image_data_format('channels_last')
#         old_data_format = 'channels_first'
#     else:
#         old_data_format = None

#     # Determine proper input shape
# #     input_shape = _obtain_input_shape(input_shape,
# #                                       default_size=299,
# #                                       min_size=71,
# #                                       data_format=K.image_data_format(),
# #                                       include_top=include_top)

#     if input_tensor is None:
#         img_input = Input(shape=input_shape)
#     else:
#         if not K.is_keras_tensor(input_tensor):
#             img_input = Input(tensor=input_tensor, shape=input_shape)
#         else:
#             img_input = input_tensor

#     x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, name='block1_conv1')(img_input)
#     x = BatchNormalization(name='block1_conv1_bn')(x)
#     x = Activation('relu', name='block1_conv1_act')(x)
#     x = Conv2D(64, (3, 3), use_bias=False, name='block1_conv2')(x)
#     x = BatchNormalization(name='block1_conv2_bn')(x)
#     x = Activation('relu', name='block1_conv2_act')(x)

#     residual = Conv2D(128, (1, 1), strides=(2, 2),
#                       padding='same', use_bias=False)(x)
#     residual = BatchNormalization()(residual)

#     x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv1')(x)
#     x = BatchNormalization(name='block2_sepconv1_bn')(x)
#     x = Activation('relu', name='block2_sepconv2_act')(x)
#     x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv2')(x)
#     x = BatchNormalization(name='block2_sepconv2_bn')(x)

#     x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block2_pool')(x)
#     x = layers.add([x, residual])

#     residual = Conv2D(256, (1, 1), strides=(2, 2),
#                       padding='same', use_bias=False)(x)
#     residual = BatchNormalization()(residual)

#     x = Activation('relu', name='block3_sepconv1_act')(x)
#     x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv1')(x)
#     x = BatchNormalization(name='block3_sepconv1_bn')(x)
#     x = Activation('relu', name='block3_sepconv2_act')(x)
#     x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv2')(x)
#     x = BatchNormalization(name='block3_sepconv2_bn')(x)

#     x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block3_pool')(x)
#     x = layers.add([x, residual])

#     residual = Conv2D(728, (1, 1), strides=(2, 2),
#                       padding='same', use_bias=False)(x)
#     residual = BatchNormalization()(residual)

#     x = Activation('relu', name='block4_sepconv1_act')(x)
#     x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv1')(x)
#     x = BatchNormalization(name='block4_sepconv1_bn')(x)
#     x = Activation('relu', name='block4_sepconv2_act')(x)
#     x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv2')(x)
#     x = BatchNormalization(name='block4_sepconv2_bn')(x)

#     x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block4_pool')(x)
#     x = layers.add([x, residual])

#     for i in range(8):
#         residual = x
#         prefix = 'block' + str(i + 5)

#         x = Activation('relu', name=prefix + '_sepconv1_act')(x)
#         x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv1')(x)
#         x = BatchNormalization(name=prefix + '_sepconv1_bn')(x)
#         x = Activation('relu', name=prefix + '_sepconv2_act')(x)
#         x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv2')(x)
#         x = BatchNormalization(name=prefix + '_sepconv2_bn')(x)
#         x = Activation('relu', name=prefix + '_sepconv3_act')(x)
#         x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv3')(x)
#         x = BatchNormalization(name=prefix + '_sepconv3_bn')(x)

#         x = layers.add([x, residual])

#     residual = Conv2D(1024, (1, 1), strides=(2, 2),
#                       padding='same', use_bias=False)(x)
#     residual = BatchNormalization()(residual)

#     x = Activation('relu', name='block13_sepconv1_act')(x)
#     x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block13_sepconv1')(x)
#     x = BatchNormalization(name='block13_sepconv1_bn')(x)
#     x = Activation('relu', name='block13_sepconv2_act')(x)
#     x = SeparableConv2D(1024, (3, 3), padding='same', use_bias=False, name='block13_sepconv2')(x)
#     x = BatchNormalization(name='block13_sepconv2_bn')(x)

#     x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block13_pool')(x)
#     x = layers.add([x, residual])

#     x = SeparableConv2D(1536, (3, 3), padding='same', use_bias=False, name='block14_sepconv1')(x)
#     x = BatchNormalization(name='block14_sepconv1_bn')(x)
#     x = Activation('relu', name='block14_sepconv1_act')(x)

#     x = SeparableConv2D(2048, (3, 3), padding='same', use_bias=False, name='block14_sepconv2')(x)
#     x = BatchNormalization(name='block14_sepconv2_bn')(x)
#     x = Activation('relu', name='block14_sepconv2_act')(x)

#     if include_top:
#         x = GlobalAveragePooling2D(name='avg_pool')(x)
#         x = Dense(classes, activation='softmax', name='predictions')(x)
#     else:
#         if pooling == 'avg':
#             x = GlobalAveragePooling2D()(x)
#         elif pooling == 'max':
#             x = GlobalMaxPooling2D()(x)

#     # Ensure that the model takes into account
#     # any potential predecessors of `input_tensor`.
#     if input_tensor is not None:
#         inputs = tf.keras.utils.get_source_inputs(input_tensor)
#     else:
#         inputs = img_input
#     # Create model.
#     model = Model(inputs, x, name='xception')

#     # load weights
#     if weights == 'imagenet':
#         if include_top:
#             weights_path = get_file('xception_weights_tf_dim_ordering_tf_kernels.h5',
#                                     TF_WEIGHTS_PATH,
#                                     cache_subdir='models')
#         else:
#             weights_path = get_file('xception_weights_tf_dim_ordering_tf_kernels_notop.h5',
#                                     TF_WEIGHTS_PATH_NO_TOP,
#                                     cache_subdir='models')
#         model.load_weights(weights_path)

#     if old_data_format:
#         K.set_image_data_format(old_data_format)
#     return model

In [28]:
# from __future__ import print_function

# import numpy as np
# import warnings

# from keras.layers import merge, Input
# from keras.layers import Dense, Activation, Flatten
# from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
# from keras.layers import BatchNormalization
# from keras.models import Model
# from keras.preprocessing import image
# import keras.backend as K

# from keras.utils.data_utils import get_file
# from keras.applications.imagenet_utils import decode_predictions, preprocess_input


# TH_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels.h5'
# TF_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
# TH_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_th_dim_ordering_th_kernels_notop.h5'
# TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'


# def identity_block(input_tensor, kernel_size, filters, stage, block):
#     '''The identity_block is the block that has no conv layer at shortcut
#     # Arguments
#         input_tensor: input tensor
#         kernel_size: defualt 3, the kernel size of middle conv layer at main path
#         filters: list of integers, the nb_filters of 3 conv layer at main path
#         stage: integer, current stage label, used for generating layer names
#         block: 'a','b'..., current block label, used for generating layer names
#     '''
#     nb_filter1, nb_filter2, nb_filter3 = filters
#     if K.image_dim_ordering() == 'tf':
#         bn_axis = 3
#     else:
#         bn_axis = 1
#     conv_name_base = 'res' + str(stage) + block + '_branch'
#     bn_name_base = 'bn' + str(stage) + block + '_branch'

#     x = Convolution2D(nb_filter1, 1, 1, name=conv_name_base + '2a')(input_tensor)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
#     x = Activation('relu')(x)

#     x = Convolution2D(nb_filter2, kernel_size, kernel_size,
#                       border_mode='same', name=conv_name_base + '2b')(x)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
#     x = Activation('relu')(x)

#     x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c')(x)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

#     x = merge([x, input_tensor], mode='sum')
#     x = Activation('relu')(x)
#     return x


# def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
#     '''conv_block is the block that has a conv layer at shortcut
#     # Arguments
#         input_tensor: input tensor
#         kernel_size: defualt 3, the kernel size of middle conv layer at main path
#         filters: list of integers, the nb_filters of 3 conv layer at main path
#         stage: integer, current stage label, used for generating layer names
#         block: 'a','b'..., current block label, used for generating layer names
#     Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
#     And the shortcut should have subsample=(2,2) as well
#     '''
#     nb_filter1, nb_filter2, nb_filter3 = filters
#     if K.image_dim_ordering() == 'tf':
#         bn_axis = 3
#     else:
#         bn_axis = 1
#     conv_name_base = 'res' + str(stage) + block + '_branch'
#     bn_name_base = 'bn' + str(stage) + block + '_branch'

#     x = Convolution2D(nb_filter1, 1, 1, subsample=strides,
#                       name=conv_name_base + '2a')(input_tensor)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
#     x = Activation('relu')(x)

#     x = Convolution2D(nb_filter2, kernel_size, kernel_size, border_mode='same',
#                       name=conv_name_base + '2b')(x)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
#     x = Activation('relu')(x)

#     x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c')(x)
#     x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

#     shortcut = Convolution2D(nb_filter3, 1, 1, subsample=strides,
#                              name=conv_name_base + '1')(input_tensor)
#     shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

#     x = merge([x, shortcut], mode='sum')
#     x = Activation('relu')(x)
#     return x


# def ResNet50(include_top=True, weights='imagenet',
#              input_tensor=None):
#     '''Instantiate the ResNet50 architecture,
#     optionally loading weights pre-trained
#     on ImageNet. Note that when using TensorFlow,
#     for best performance you should set
#     `image_dim_ordering="tf"` in your Keras config
#     at ~/.keras/keras.json.
#     The model and the weights are compatible with both
#     TensorFlow and Theano. The dimension ordering
#     convention used by the model is the one
#     specified in your Keras config file.
#     # Arguments
#         include_top: whether to include the 3 fully-connected
#             layers at the top of the network.
#         weights: one of `None` (random initialization)
#             or "imagenet" (pre-training on ImageNet).
#         input_tensor: optional Keras tensor (i.e. xput of `layers.Input()`)
#             to use as image input for the model.
#     # Returns
#         A Keras model instance.
#     '''
#     if weights not in {'imagenet', None}:
#         raise ValueError('The `weights` argument should be either '
#                          '`None` (random initialization) or `imagenet` '
#                          '(pre-training on ImageNet).')
#     # Determine proper input shape
    
#     input_shape = (224, 224, 3)

#     if input_tensor is None:
#         img_input = Input(shape=input_shape)
#     else:
#         if not K.is_keras_tensor(input_tensor):
#             img_input = Input(tensor=input_tensor)
#         else:
#             img_input = input_tensor
#     if K.image_dim_ordering() == 'tf':
#         bn_axis = 3
#     else:
#         bn_axis = 1

#     x = ZeroPadding2D((3, 3))(img_input)
#     x = Convolution2D(64, 7, 7, subsample=(2, 2), name='conv1')(x)
#     x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
#     x = Activation('relu')(x)
#     x = MaxPooling2D((3, 3), strides=(2, 2))(x)

#     x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
#     x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
#     x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

#     x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
#     x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
#     x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
#     x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

#     x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
#     x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
#     x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
#     x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
#     x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
#     x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

#     x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
#     x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
#     x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

#     x = AveragePooling2D((7, 7), name='avg_pool')(x)

#     if include_top:
#         x = Flatten()(x)
#         x = Dense(1000, activation='softmax', name='fc1000')(x)

#     model = Model(img_input, x)

#     # load weights
#     if weights == 'imagenet':
#         print('K.image_dim_ordering:', K.image_dim_ordering())
#         if K.image_dim_ordering() == 'th':
#             if include_top:
#                 weights_path = get_file('resnet50_weights_th_dim_ordering_th_kernels.h5',
#                                         TH_WEIGHTS_PATH,
#                                         cache_subdir='models',
#                                         md5_hash='1c1f8f5b0c8ee28fe9d950625a230e1c')
#             else:
#                 weights_path = get_file('resnet50_weights_th_dim_ordering_th_kernels_notop.h5',
#                                         TH_WEIGHTS_PATH_NO_TOP,
#                                         cache_subdir='models',
#                                         md5_hash='f64f049c92468c9affcd44b0976cdafe')
#             model.load_weights(weights_path)
#             if K.backend() == 'tensorflow':
#                 warnings.warn('You are using the TensorFlow backend, yet you '
#                               'are using the Theano '
#                               'image dimension ordering convention '
#                               '(`image_dim_ordering="th"`). '
#                               'For best performance, set '
#                               '`image_dim_ordering="tf"` in '
#                               'your Keras config '
#                               'at ~/.keras/keras.json.')
#                 convert_all_kernels_in_model(model)
#         else:
#             if include_top:
#                 weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
#                                         TF_WEIGHTS_PATH,
#                                         cache_subdir='models',
#                                         md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
#             else:
#                 weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
#                                         TF_WEIGHTS_PATH_NO_TOP,
#                                         cache_subdir='models',
#                                         md5_hash='a268eb855778b3df3c7506639542a6af')
#             model.load_weights(weights_path)
#     return model

In [29]:
# model2 = VGG19(include_top=False,
#              weights='imagenet')

In [30]:
# x = model2.output
# # x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)

# predictions = Dense(6, activation='softmax')(x)


# model2 = Model(inputs=model2.input, outputs=predictions)

In [31]:
# model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # train the model on the new data for a few epochs
# init_time = datetime.datetime.now()
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 1, verbose=1, factor=0.3, min_lr=0.000001)

# trained = model2.fit(
#                     train_images, train_labels,
#                     validation_data = (val_images, val_labels),
#                     batch_size = BATCH_SIZE, 
#                     epochs=14,
#                     callbacks=[learning_rate_reduction])